Ejercicios
1.-Cree un RDD llamado lenguajes que contenga los siguientes lenguajes de programación: Python, R, C, Scala, Rugby y SQL.
    a).-Obtenga un nuevo RDD a partir del RDD lenguajes donde todos los lenguajes de programación estén en mayúsculas.
    b).-Obtenga un nuevo RDD a partir del RDD lenguajes donde todos los lenguajes de programación estén en minúsculas.
    c).-Cree un nuevo RDD que solo contenga aquellos lenguajes de programación que comiencen con la letra R.
2.-Cree un RDD llamado pares que contenga los números pares existentes en el intervalo [20;30].
    a).-Cree el RDD llamado sqrt, este debe contener la raíz cuadrada de los elementos que componen el RDD pares.
    b).-Obtenga una lista compuesta por los números pares en el intervalo [20;30] y sus respectivas raíces cuadradas. Un ejemplo del resultado deseado para el intervalo [50;60] sería la lista [50, 7.0710678118654755, 52, 7.211102550927978, 54, 7.3484692283495345, 56, 7.483314773547883, 58, 7.615773105863909, 60, 7.745966692414834].
    c).-Eleve el número de particiones del RDD sqrt a 20.
    d).-Si tuviera que disminuir el número de particiones luego de haberlo establecido en 20, ¿qué función utilizaría para hacer más eficiente su código?
3.-Cree un RDD del tipo clave valor a partir de los datos adjuntos como recurso a esta lección. Tenga en cuenta que deberá procesar el RDD leído para obtener el resultado solicitado. Supongamos que el RDD resultante de tipo clave valor refleja las transacciones realizadas por número de cuentas. Obtenga el monto total por cada cuenta.
**Tip: Cree su propia función para procesar el RDD leído.**

In [2]:
import findspark 
from pyspark.sql import SparkSession

findspark.init()
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [3]:
#punto 1
rdd = sc.parallelize(['Python', 'R', 'C', 'Scala', 'Rugby','SQL'])

In [4]:
#punto a)
rdd_mayus = rdd.map(lambda x: x.upper())
rdd_mayus.collect()

['PYTHON', 'R', 'C', 'SCALA', 'RUGBY', 'SQL']

In [5]:
#punto b)
rdd_minus = rdd.map(lambda x: x.lower())
rdd_minus.collect()

['python', 'r', 'c', 'scala', 'rugby', 'sql']

In [7]:
#punto c
rdd_r = rdd.filter(lambda x: x.startswith('R'))
rdd_r.collect()

['R', 'Rugby']

In [17]:
#punto 2
rdd_pares = sc.parallelize(range(20,31,2))
rdd_pares.collect()

[20, 22, 24, 26, 28, 30]

In [23]:
#a)
import math
rdd_raiz = rdd_pares.map(lambda x: math.sqrt(x))
rdd_raiz.collect()

[4.47213595499958,
 4.69041575982343,
 4.898979485566356,
 5.0990195135927845,
 5.291502622129181,
 5.477225575051661]

In [40]:
#b)
import math
rdd_compuesto = sc.parallelize(range(20,31,2))
rdd_raiz2 = rdd_compuesto.flatMap(lambda x: (x,math.sqrt(x))).collect()
print(rdd_raiz2)


[20, 4.47213595499958, 22, 4.69041575982343, 24, 4.898979485566356, 26, 5.0990195135927845, 28, 5.291502622129181, 30, 5.477225575051661]


In [35]:
#c)
rdd_partition = rdd_raiz2.repartition(20)
rdd_partition.getNumPartitions()

20

In [36]:
#d)
rdd_coalesce = rdd_partition.coalesce(10)
rdd_coalesce.getNumPartitions()

10

In [47]:
#3
rdd_texto = sc.textFile('./proceso.txt/')
rdd_texto.collect()


['(1001, 52.3)',
 '(1005, 20.8)',
 '(1001, 10.1)',
 '(1004, 52.7)',
 '(1005, 20.7)',
 '(1002, 85.3)',
 '(1004, 20.9)']

In [50]:
def proceso(archivo):
    return (tuple(archivo.replace('(',' ').replace(')',' ').split(',')))

rdd_llave_valor = rdd_texto.map(proceso)
rdd_llave_valor.collect()

[(' 1001', ' 52.3 '),
 (' 1005', ' 20.8 '),
 (' 1001', ' 10.1 '),
 (' 1004', ' 52.7 '),
 (' 1005', ' 20.7 '),
 (' 1002', ' 85.3 '),
 (' 1004', ' 20.9 ')]

In [54]:
rdd_llave_valor.reduceByKey(lambda x,y: float(x) + float(y)).collect()

[(' 1005', 41.5), (' 1004', 73.6), (' 1001', 62.4), (' 1002', ' 85.3 ')]